In [ ]:
from dataset import CityscapesDataset

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1
